In [5]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
from joblib import load

# Cargar el modelo y el escalador
try:
    modelo = load('random_forest_model.pkl')  # Asegúrate de que el archivo esté en el mismo directorio
    scaler = load('random_forest_scaler.pkl')
except Exception as e:
    raise RuntimeError(f"Error cargando modelo o escalador: {str(e)}")

# Inicializar la aplicación FastAPI
app = FastAPI()

# Clase para definir la estructura de los datos de entrada
class InputData(BaseModel):
    features: list[float]  # Lista de valores para las características

# Ruta de prueba
@app.get("/")
def home():
    return {"mensaje": "API funcionando correctamente"}

# Ruta para predecir
@app.post("/predict/")
def predict(data: InputData):
    try:
        # Convertir la lista de entrada a un array de NumPy
        input_features = np.array(data.features).reshape(1, -1)

        # Escalar los datos con el escalador
        scaled_features = scaler.transform(input_features)

        # Realizar la predicción
        prediction = modelo.predict(scaled_features)
        probability = modelo.predict_proba(scaled_features)

        # Devolver los resultados
        return {
            "prediction": int(prediction[0]),  # Resultado de la predicción
            "probability": probability.tolist()  # Probabilidades asociadas
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error al procesar la predicción: {str(e)}")


In [6]:
app